# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [1]:
import requests
from tqdm import tqdm
import pandas as pd
import pickle
import sqlite3 as db
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_columns', None)

## Список объектов

In [2]:
# Скачиваем первый батч из 100 обьектов недвижимости и получаем total_obj - общее количество обьектов
objids = []
offset_ = 0
limit_ = 100
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()
total_obj = objects_data.get('data').get('total') #общее количество обьектов
objids = [x.get('objId') for x in objects_data.get('data').get('list')] #начинаем формировать список id обьектов

# собираем все id обьектов
for i in tqdm(range(total_obj//100)):
    offset_ += limit_
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    objects_data = res.json()
    objids.extend([x.get('objId') for x in objects_data.get('data').get('list')])

100%|████████████████████████████████████████████████████████████████████████████████| 106/106 [02:01<00:00,  1.14s/it]


In [9]:
# скачиваем описание обьектов и добавляем их в DataFrame

df = pd.DataFrame()
test_id = list(pd.Series(objids).sample(100)) #100 случайных обьектов для тестов
for obj in tqdm(objids):
    url_obj = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{obj}'
    res = requests.get(url_obj)
    df = pd.concat([df, pd.json_normalize(res.json().get('data'),
                                          max_level=3)], 
                    axis=0)

100%|████████████████████████████████████████████████████████████████████████████| 10672/10672 [40:57<00:00,  4.34it/s]


In [65]:
url_obj = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/13383'
res = requests.get(url_obj)
data = res.json().get('data')

In [67]:
data

{'id': 13383,
 'pdId': 28105,
 'developer': {'devId': 306,
  'devShortCleanNm': '2МЕН ГРУПП',
  'devShortNm': '2МЕН ГРУПП',
  'devFullCleanNm': '2МЕН ГРУПП',
  'problObjCnt': 0,
  'buildObjCnt': 5,
  'comissObjCnt': 0,
  'regRegionDesc': 'Тюменская область',
  'devPhoneNum': '+7(345)279-18-88',
  'devSite': '2mengroup.ru',
  'devEmail': 'info@2mengroup.ru',
  'devInn': '7701651356',
  'devOgrn': '1067746424899',
  'devKpp': '720301001',
  'devLegalAddr': 'Тюменская область, город Тюмень, улица Комсомольская дом 75/5',
  'devFactAddr': 'Тюменская область, город Тюмень, улица Комсомольская 75/5',
  'lastRpdId': 632865,
  'fundGuarantyFlg': 1,
  'devOrgRegRegionCd': 72,
  'devEmplMainFullNm': 'Киселев Сергей Михайлович',
  'developerGroupName': '2МЕН ГРУПП ДЕВЕЛОПМЕНТ',
  'orgForm': {'id': 1, 'fullForm': 'Акционерное общество', 'shortForm': 'АО'},
  'companyGroupId': 5776,
  'objGuarantyEscrowFlg': 0,
  'govFundFlg': 0},
 'region': 72,
 'address': 'г Тюмень',
 'floorMin': 4,
 'floorMax': 

In [68]:
df.info(max_cols=200)

In [11]:
df.to_excel('HW1_objects_data.xlsx', index=False)

In [13]:
df.to_pickle('HW1_objects_data.pkl', compression='zip')

In [56]:
# сохраним датафпейм в таблицу sqlite3
conn = db.connect(":memory:") 
curr = conn.cursor()

#для простоты уберем столбцы со списками
df_for_sql = df.drop(['objectTransportInfo', "photoRenderDTO", "metro.colors"], axis = 1)

#добавим в записываемый df строку с именами колонок
df_for_sql = pd.concat([pd.DataFrame([df_for_sql.columns.values], columns=df_for_sql.columns), df_for_sql], ignore_index=True)

# на основе датафрейма df создем таблицу в sqlite3 с именем table1
df_for_sql.to_sql(name='table1', con=conn, if_exists='replace', index=False, index_label=True)

In [58]:
# проверяем, что select работает
result = curr.execute('select * from table1')
df_from_sql = pd.DataFrame(result)
df_from_sql.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105
0,id,pdId,region,address,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,objLkFinishTypeDesc,objLkFreePlanDesc,objElemParkingCnt,objSquareLiving,rpdNum,rpdPdfLink,rpdIssueDttm,objLkLatitude,objLkLongitude,objCompensFundFlg,objProblemFlg,objLkClassDesc,loadDttm,objGuarantyEscrowFlg,objectType,residentialBuildings,newBuildingId,objFlatCnt,objFlatSq,objNonlivElemCnt,objStatus,isAvailableWantLiveHere,objInfrstrBicycleLaneFlg,objInfrstrRampFlg,objInfrstrCurbLoweringFlg,objElevatorCargoPassengerCnt,conclusion,objGreenHouseFlg,objEnergyEffShortDesc,infrastructureIndexValue,developer.devId,developer.devShortCleanNm,developer.devShortNm,developer.devFullCleanNm,developer.problObjCnt,developer.buildObjCnt,developer.comissObjCnt,developer.regRegionDesc,developer.devPhoneNum,developer.devSite,developer.devEmail,developer.devInn,developer.devOgrn,developer.devKpp,developer.devLegalAddr,developer.devFactAddr,developer.lastRpdId,developer.fundGuarantyFlg,developer.devOrgRegRegionCd,developer.devEmplMainFullNm,developer.orgForm.id,developer.orgForm.fullForm,developer.orgForm.shortForm,developer.objGuarantyEscro...,developer.govFundFlg,floorMin,floorMax,miniUrl,objFloorCnt,objTransferPlanDt,objLivCeilingHeight,objInfrstrPlaygrndCnt,objInfrstrSportGroundCnt,objInfrstrTrashAreaCnt,objInfrstrObjPrkngCnt,objInfrstrNotObjPrkngCnt,objElevatorWheelchairCnt,objElevatorPassengerCnt,objElevatorCargoCnt,soldOutPerc,objPriceAvg,generalContractorNm,nonlivFirstFloor,objLkSaleDepEmail,developer.developerGroupName,developer.companyGroupId,quartography.objLivElem1KCnt,quartography.objLivElem2KCnt,quartography.objLivElem3KCnt,quartography.objLivElem4KCnt,quartography.objLivElemCnt,quartography.objLivElemEnt...,quartography.objLivElemEnt...,transportDistIndex,transportDistIndexValue,nameObj,greenAreaIndexValue,airQualityIndexValue,complexShortNm,developer.bankruptStage.ba...,developer.bankruptStage.ba...,developer.bankruptStage.ba...,developer.orgBankruptMsgDttm,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk
1,29679,124647,61,г Таганрог,18,2020-03-31,Другое,Под ключ,Есть,0,1519.0,61-000818,https://xn--80az8a.xn--d1a...,20-03-2019 18:41,47.2238,38.8513,0,0,Комфорт,02-10-2022 01:32,0,normal,1,40800,18,1519.0,0,0,1,0,0,0,0,0,0,Не нормируется,7,7424,212-2,212-2,212-2,0,1,0,Ростовская область,+7(961)285-90-10,taganrogjsk.wixsite.com,kirpol76@gmail.com,6154572922,1116154005923,615401001,"Ростовская область, город ...",обл Ростовская город Таган...,126093,0,61.0,Снимщикова Ольга Викторовна,20,Жилищный или жилищно-строи...,ЖСК,0,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,13383,28105,72,г Тюмень,68,2022-12-31,Другое,По желанию заказчика,Есть,31,6293.78,72-000095,https://xn--80az8a.xn--d1a...,07-09-2022 15:16,57.1519,65.5536,1,0,Элитный,02-10-2022 01:32,0,normal,1,39024,68,6293.78,10,0,1,0,1,1,7,0,0,B,10,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,Тюменская область,+7(345)279-18-88,2mengroup.ru,info@2mengroup.ru,7701651356,1067746424899,720301001,"Тюменская область, город Т...","Тюменская область, город Т...",632865,1,72.0,Киселев Сергей Михайлович,1,Акционерное общество,АО,0,0,4.0,20.0,https://xn--80az8a.xn--d1a...,20.0,28-02-2023 12:00,"2,8",1.0,1.0,1.0,0.0,11.0,0.0,5.0,1.0,0.407163898325013,53109.0,ООО Транспромжилстрой-2005...,1.0,info@2mengroup.ru,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,5776.0,13.0,21.0,34.0,0.0,68.0,1.0,6.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


# HomeWork_2
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф" ++++
* Cохранить результаты в pandas dataframe ++++
<hr>

* Проверить состояние датафрейма и привести его в формат, позволяющий дальнейшее исследование данных:
-- Привести колонки датафрейма к одному типу данных ++++
-- Проверить индекс, чтобы он соответствовал порядку строк ++++
-- Проверить наличие пропусков и в случае обнаружения, определить стратегию их обработки ++++
<hr>

In [2]:
# считываем датафрейм и оставляем только необходимые для исследований поля

col = ['id', 'region', 'objReady100PercDt', 'objFloorCnt', 'objFlatCnt', 'objElemLivingCnt',
       'objSquareLiving', 'objPriceAvg', 'objElemParkingCnt'] 

df = pd.read_pickle('HW1_objects_data.pkl', compression='zip').iloc[1:].reset_index(drop=True)[col]
df = df.query('objFloorCnt != 0')


colmap = {'objReady100PercDt':'YearReady', 'objElemLivingCnt':'ElemCnt', 'objSquareLiving':'SqrLiv',
         'objElemParkingCnt':'PrkCnt'}
df.rename(mapper=colmap, inplace=True, axis=1)
df['YearReady'] = df['YearReady'].astype('datetime64[ns]').dt.year
df['objFloorCnt'] = df['objFloorCnt'].astype('int64')


In [3]:
#скачаем названия регионов и добавим в df

# url_region = 'https://github.com/hflabs/region/raw/master/region.csv'
# res = requests.get(url_region)
# data_region = res.text
# with open('region.csv', 'w+', encoding='utf-8') as file:
#     file.write(data_region)

df_region = pd.read_csv('region.csv', sep=',')[['name_with_type', 'kladr_id']]
df_region['kladr_id'] = (df_region['kladr_id']/100000000000).astype('int64')
df_region.set_index('kladr_id', inplace=True)

df = pd.merge(df, df_region, left_on='region', right_on=df_region.index)
df.rename(columns={'name_with_type':'name_region'}, inplace=True)
df['region'] = df['region'].astype('str')
df.sample(3)

,id,region,YearReady,objFloorCnt,objFlatCnt,ElemCnt,SqrLiv,objPriceAvg,PrkCnt,name_region
7884,47937,14,2023,6,142,142,6124.44,NaN,0,Респ Саха /Якутия/
5374,8930,78,2024,1,0,0,0.00,NaN,0,г Санкт-Петербург
5851,42900,78,2024,12,538,538,45000.01,265728.0,326,г Санкт-Петербург


In [4]:
# для исследования средней цены оставим обьекты с известной ценой квадратного метра
df_price = df[df['objPriceAvg'].notnull()]
df_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7520 entries, 0 to 10656
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           7520 non-null   int64  
 1   region       7520 non-null   object 
 2   YearReady    7520 non-null   int64  
 3   objFloorCnt  7520 non-null   int64  
 4   objFlatCnt   7520 non-null   int64  
 5   ElemCnt      7520 non-null   int64  
 6   SqrLiv       7520 non-null   float64
 7   objPriceAvg  7520 non-null   float64
 8   PrkCnt       7520 non-null   int64  
 9   name_region  7520 non-null   object 
dtypes: float64(2), int64(6), object(2)
memory usage: 646.2+ KB


## Исследование

* Провести мини-исследование рынка строящейся недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов.
-- Рассчитать средние цены за квадратный метр по регионам. Важно чтобы пропуски не учитывались при расчете средней цены. ++++
-- Рассчитать объемы вводимого жилья по регионам и по годам.
-- Сравнить этажность домов, количество квартир, жилую площадь, количество парковочных мест и цену за квадратный метр в разных регионах
-- Визуализировать разброс, связь и распределение этих показателей по регионам
-- Сделать выводы по каждому из расчетов и графиков

In [5]:
# !pip install cufflinks
# !pip install plotly
# Standard plotly imports
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks as cf
cf.go_offline(connected=True)
init_notebook_mode(connected=True)

In [6]:
# столбчатый график средней цены кв.м. в регионе
mean_sqr_mtr = df_price.groupby('name_region')['objPriceAvg'].agg('mean').sort_values(ascending=False)
mean_sqr_mtr.iplot(kind='bar', xTitle='Регион РФ',
                        yTitle='средняя цена', title='Cредняя цена за квадратный метр по регионам, в рублях'
                       )

# сильно в отрыве Москва.
# Сахалин и Приморский край делят 2-5 места с МО и ЛО.
# Самое дешевое жилье в Дагестане и Чечне

In [15]:
# Рассчитать объемы вводимого жилья по регионам и по годам.

df_SqrLiv = df.groupby(['name_region', 'YearReady'])['SqrLiv'].agg('sum').reset_index().sort_values(by=['YearReady', 'SqrLiv'])
year_list = sorted(df['YearReady'].unique())
region_list = ['all']+sorted(df_SqrLiv['name_region'].unique())

@interact_manual
def bar_plot(year_from = year_list, year_to = year_list, region=region_list):
    if region == 'all':        
        query = 'YearReady >= @year_from and YearReady <= @year_to'
    else:
        query = 'YearReady >= @year_from and YearReady <= @year_to and name_region==@region'
    fig = px.bar(df_SqrLiv.query(query),
                 y="name_region",
                 x="SqrLiv",
                 color='YearReady',
                 title=f"Обьем вводимого жилья по регионам c {year_from} по {year_to} год.",
                orientation='h')
#     fig.update_layout(autosize=False)
    fig.show()
    
# Вывод: больше всего жилья строят в столицах и их областях, а также Краснодарском крае
# ну и ожидаемо 23, 24 года максимальные по обьему вводимого жилья

interactive(children=(Dropdown(description='year_from', options=(2017, 2018, 2019, 2020, 2021, 2022, 2023, 202…

In [17]:
# Сравнить этажность домов, количество квартир, жилую площадь,
# количество парковочных мест и цену за квадратный метр в разных регионах


map_param = {'objFloorCnt':'кол-ва этажей в доме', 'objFlatCnt':'кол-ва квартир в доме',
             'objPriceAvg':'средней цены кв.м дома', 'SqrLiv':'жилой площади дома', 'PrkCnt':'кол-ва парковочных мест в доме'}
list_of_param = map_param.keys()
id_regions = df.region.unique()

@interact_manual
def box_plot(x=list_of_param, region=region_list, log_x=True):
    
    #используем в анализе только ненулевые значения параметров
    # а в случае цены исследуем df_price (фрейм с не пустыми ценами)
    data = df_price if x == 'objPriceAvg' else df[df[f'{x}'] >= 1]
    
    # готовим запрос для фильтрации по региону
    query = 'index' if region == 'all' else 'name_region == @region'
    
    if region=='all':
        fig = px.box(data.query(query), y='name_region', x=x, log_x=log_x,
                     title=f'График распределения {map_param.get(x)} от региона',
                     orientation='h', hover_data=['region'])
        fig.update_layout(autosize=False, width=800, height=1300)
    else:
        fig = px.histogram(data.query('name_region == @region'), x=x,
                     title=f'График распределения {map_param.get(x)} в регионе: {region}')
        
        
    fig.show()

interactive(children=(Dropdown(description='x', options=('objFloorCnt', 'objFlatCnt', 'objPriceAvg', 'SqrLiv',…

In [27]:
# Визуализировать разброс, связь и распределение этих показателей по регионам

@interact_manual
def scatter_plot(x= list_of_param, y=list_of_param, log_x=[True, False], log_y=[True, False]):
    fig = px.scatter(df_price, y=y, x=x, color='name_region', log_x=log_x, log_y=log_y)
    fig.show()
    
# выводы: в Мск высотные дома стоят от 200-500тыс.р, также у этих домов больше всего парковочных мест.
# Парковок много в новостройках Краснодарского края, и не только в дорогих домах.
# Большая плотность обьектов с ценой 100-200к и площадью от 7 до 35тыс кв м в МО



interactive(children=(Dropdown(description='x', options=('objFloorCnt', 'objFlatCnt', 'objPriceAvg', 'SqrLiv',…